# Discrete HMMs

## Agenda
* Key ideas and words
* HMM Overview
  * What confused me
* How I'm approaching this
* Going through examples
  * State estimation
  * Parameter estimation
* Where I would use Dynamax in my past career
  * Practical considerations
* Updated Book Club Focus

## References
* This repo https://github.com/canyon289/ssm_book_club
*  Casino HMMs from Dynamax
    *  https://probml.github.io/dynamax/notebooks/hmm/casino_hmm_inference.html
    * https://probml.github.io/dynamax/notebooks/hmm/casino_hmm_learning.html
*  Complete reading of https://nipunbatra.github.io/hmm/


# Key ideas and words

## Discrete HMM

$$
\begin{align}
p(y_{1:T}, z_{1:T} \mid \theta) 
&= \overbrace{\mathrm{Cat}(z_1 \mid \pi)}^{Prior for Initial State}
\underbrace{\prod_{t=2}^T \mathrm{Cat}(z_t \mid A_{z_{t-1}})}_{Transition Model}
\overbrace{\prod_{t=1}^T \mathrm{Cat}(y_t \mid B_{z_t})}^{Observation Model}
\end{align}
$$


## Things that were challenging to me
* Change of symbols
  * Duplication of symbols in same text
  * Python variable names that correlated to these terms
* Differing usage of words
* Ambiguous terms
  * Especially in the code
* Abstractions in Dynamax
  * Required lots of tracing

## Time dependencies
* Things that are independent of time
  * Transition Matrix*
  * Prior Probability
  * Emission Prob assuming state
  
* Things dependent on a particular time window but independent of others
  * Observations
  * Log likelihood

* Things that change over time
  * State estimation
  * Emission probability after updating state probability


## Terminology
* Forward Filter
* Prediction - Estimating th next time step
* Smoothing
  * Forward Backward Pass
* Viterbi Algorithm
  * Sequence Assignment

## Symbols

**Insert Symbols

## Applied Example, Filtering, Smoothing,  - Biased coin toss

In [6]:
import jax.numpy as jnp
import jax.random as jr
# import matplotlib.pyplot as plt
from jax import vmap


In [2]:
import dynamax
from dynamax.hidden_markov_model import CategoricalHMM


This a modified version of dynamax for the HMM Book club session
Code can be found here https://github.com/canyon289/dynamax/tree/hmm_session


In [7]:
initial_probs = jnp.array([0.5, 0.5])

transition_matrix = jnp.array([[1.0, 0.0],
                               [0.0, 1.0]])

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [ ]:
initial_probs = jnp.array([0.5, 0.5])

transition_matrix = jnp.array([[1.0, 0.0],
                               [0.0, 1.0]])


### Basic verification using Bayes Theorem

### Non identity transition matrix

## Computational complexity
Speedups from Jax

## Applied Example Parameter Learning

## Things I noticed in the code

## Takeaways

* Jax speedups are largely around computational 
* Newer estimation Algorithms provide some speedup as well
* Dynamax supports many features
  * Looks like more to come

## Book Club Focus